In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
from Utilities.plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import qmc
import numpy as np

In [ ]:
sampler = qmc.Sobol(d=3, scramble=True, seed=1234)

N_f = 64*50
xyz_f = sampler.random(N_f)
xyz_bc = sampler.random(100)

# 0–63 にスケーリング
xyz_f = np.array([0, 0, 0]) + np.array([63, 63, 63]) * xyz_f

# z=0 面に境界点を配置
xyz_bc_points = np.array([63, 63, 0]) * xyz_bc

# 結合
xyz_all = np.vstack([xyz_f, xyz_bc_points])

In [ ]:
import pyvista as pv

nx = ny = nz = 64
x = np.arange(nx)
y = np.arange(ny)
z = np.arange(nz)

xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')
mesh = pv.StructuredGrid(xv, yv, zv)


In [ ]:
points_f = pv.PolyData(xyz_f)
points_bc = pv.PolyData(xyz_bc_points)


pv.global_theme.notebook = True
pv.global_theme.jupyter_backend = 'static'

p = pv.Plotter(window_size=(1000, 600))

# 枠
p.show_bounds(
    grid='front',
    show_xlabels=False,
    show_ylabels=False,
    show_zlabels=False,
)
p.add_mesh(mesh.outline(), color='k')

# Sobol 内点（青）
p.add_mesh(
    points_f,
    color='blue',
    point_size=7,
    render_points_as_spheres=True,
    opacity=0.6
)

# 境界点（赤）
p.add_mesh(
    points_bc,
    color='red',
    point_size=9,
    render_points_as_spheres=True
)

# カメラ設定（流線と同じ）
p.camera_position = "xy"
p.camera.roll = -30
p.camera.elevation = -70
p.camera.zoom(1.3)

p.show()
